# Advanced Querying Mongo

Importing libraries and setting up connection

In [2]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [3]:
db = client.get_database("ironhack")

In [4]:
db.list_collection_names()

['companies']

In [5]:
c = db.get_collection("companies")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [6]:
# Your Code
babelgum = list(c.find({"name": "Babelgum"}))

In [7]:
#print(babelgum)

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [8]:
# Your Code
more_5000 = {"number_of_employees": {"$gt" : 5000}}
n_employees = list(c.find(more_5000).limit(20))
#len(n_employees)

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [9]:
# Your Code
proy = {"name": 1, "founded_year": 1, "_id": 0}
cond1 = {"founded_year": {"$gte" : 2000}}
cond2 = {"founded_year": {"$lte" : 2005}}
companies = list(c.find({"$and": [cond1,cond2]},proy))

In [10]:
#companies

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [11]:
# Your Code
proy = {"name": 1, "ipo": 1, "_id": 0}
cond1 = {"founded_year": {"$lte" : 2010}}
cond2 = {"ipo.valuation_amount": {"$gt": 100000000}}
companies = list(c.find({"$and": [cond1,cond2]},proy))

In [12]:
#companies

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [13]:
# Your Code
cond1 = {"number_of_employees": {"$lt" : 1000}}
cond2 = {"founded_year": {"$lt" : 2005}}
companies = list(c.find({"$and": [cond1,cond2]}).sort("number_of_employees").limit(10))

In [14]:
#companies

### 6. All the companies that don't include the `partners` field.

In [15]:
# Your Code
companies = list(c.find({"partners": {"$exists": False}}))

In [16]:
len(companies) #this makes sense because there are 18.801 documents. If we do True, that´s the len.

0

### 7. All the companies that have a null type of value on the `category_code` field.

In [17]:
# Your Code
companies = list(c.find({"category_code": {"$type": "null" }}))

In [18]:
comp2 = list(c.find({"category_code": None}))

In [19]:
companies == comp2

True

In [20]:
#companies

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [21]:
# Your Code
proy = {"name": 1, "employees": 1, "_id": 0}
cond1 = {"number_of_employees": {"$gte" : 100}}
cond2 = {"number_of_employees": {"$lt" : 1000}}
companies = list(c.find({"$and": [cond1,cond2]}))

In [22]:
#companies

### 9. Order all the companies by their IPO price in a descending order.

In [23]:
# Your Code
companies = list(c.find({}).sort("ipo",-1))

In [24]:
#companies

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [25]:
# Your Code
companies = list(c.find({}).sort("number_of_employees",-1).limit(10))

In [26]:
#companies

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [27]:
# Your Code
cond1 = {"founded_month": {"$gt" : 6}}
companies = list(c.find(cond1).limit(1000))

In [28]:
#companies

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [29]:
# Your Code
cond1 = {"founded_year": {"$lt" : 2000}}
cond2 = {"acquisition.price_amount": {"$gt": 1000000}}
companies = list(c.find({"$and": [cond1,cond2]}))

In [30]:
#companies

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [31]:
# Your Code
proy = {"name": 1, "acquisition": 1, "_id": 0}
cond1 = {"acquisition.acquired_year": {"$gt": 2010}}
companies = list(c.find(cond1, proy).sort("acquisition.price_amount", -1))

In [32]:
#companies

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [33]:
# Your Code
proy = {"name": 1, "founded_year": 1, "_id": 0}
cond1 = {"founded_year": {"$ne": None}}
companies = list(c.find(cond1, proy).sort("founded_year",1))

In [34]:
#companies

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [35]:
# Your Code
cond1 = {"founded_day": {"$lte": 7}}
companies = list(c.find(cond1).sort("acquisition.price_amount", -1).limit(10))

In [36]:
#companies

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [37]:
# Your Code
cond1 = {"category_code": "web"}
cond2 = {"number_of_employees": {"$gt" : 4000}}
companies = list(c.find({"$and": [cond1,cond2]}).sort("number_of_employees",1))

In [38]:
#companies

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [39]:
# Your Code
cond1 = {"acquisition.price_amount": {"$gt": 10000000}}
cond2 = {"acquisition.price_currency_code": "EUR"}
companies = list(c.find({"$and": [cond1,cond2]}))

In [40]:
#companies

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [41]:
# Your Code
proy = {"name": 1, "acquisition": 1, "_id": 0}
cond1 = {"acquisition.acquired_month": {"$lt": 4}}
companies = list(c.find(cond1, proy).limit(10))

In [42]:
#companies

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [43]:
proy = {"name": 1, 'founded_year': 1, 'acquisition.acquired_year':1, '_id': 0}
cond1 = {'founded_year': {"$in": list(range(2000,2011))}}
cond2 = {'acquisition.acquired_year': {"$nin": list(range(0,2012))}}
list(c.find({"$and": [cond1,cond2]}, proy).limit(3))

[{'name': 'Wetpaint',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [44]:
# Your Code
query = {"$where": '(this.deadpooled_year - this.founded_year > 3)'}